# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [61]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [62]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [63]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [64]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [65]:
from cassandra.cluster import Cluster
cluster = Cluster()

# Establishing a session
session = cluster.connect()
print("Create Session")

Create Session


#### Create Keyspace

In [66]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [67]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1 
#### SQL Requirement:
<p> Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4</p>

#### Select Query: 
<p>select artist, title, lengh from song_data_by_session where sessionId=338 and itemInSession = 4</p>

- Table Name: song_data_by_session
- Primary Key: sessionId, itemInSession
- Partition Key: sessionId
- Clustering Columns: itemInSession
- Outout Cols: artist, title, lengh

In [68]:
create_table_query = "CREATE TABLE IF NOT EXISTS song_data_by_session "
create_table_query = create_table_query + "(sessionId int, itemInSession int, artist text, title text,lengh float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(create_table_query)
except Exception as e:
    print(e)

    
query = "INSERT INTO song_data_by_session (sessionId, itemInSession, artist, title, lengh)"
query = query + " VALUES (%s, %s, %s, %s, %s)"

query1 = "select artist, title, lengh from song_data_by_session where sessionId=338 and itemInSession = 4"


                    

In [69]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

- Composite Primary Key: sessionId, itemInSession
- Partition Key: sessionid
- Clustering Key: itemInSession
<br/> Partitioning is done by sessionid and within that partition, rows ordered by the itemInSession, artist, title, lengh

In [70]:
try:
    print(query1)
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.title, row.lengh)

select artist, title, lengh from song_data_by_session where sessionId=338 and itemInSession = 4
Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2 
#### SQL Requirement:
<p> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182</p>

#### Select Query: 
<p>select artist, song, itemInSession, firstName, lastName from song_user_data_by_session where userid = 10 and sessionid = 182'</p>

- Table Name: song_user_data_by_session
- Composite Primary Key: userid, sessionid, itemInSession
- Partition Key: userid, sessionid
- Clustering Columns: itemInSession
- Outout Cols: artist, song, itemInSession, firstName, lastName

In [71]:
create_table_query2 = "CREATE TABLE IF NOT EXISTS song_user_data_by_session "
    
create_table_query2 = create_table_query2 + "(userid int,sessionid int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userid, sessionid), itemInSession))"
try:
    session.execute(create_table_query2)
except Exception as e:
    print(e)

    
insert_query2 = "INSERT INTO song_user_data_by_session (userid, sessionid, itemInSession, artist, song, firstName, lastName)"
insert_query2 = insert_query2 + " VALUES (%s, %s, %s, %s, %s,%s, %s)"

query2 = "select itemInSession, artist, song, firstName, lastName from song_user_data_by_session where userid = 10 and sessionid = 182"                    

In [72]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(insert_query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

- Composite Primary Key: userid, sessionid, itemInSession
- Partition Key: userid, sessionid
- Clustering Key: itemInSession
<br/> Partitioning is done by userid, sessionid, and within that partition, rows ordered by the itemInSession

In [73]:
try:
    print(query2)
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.iteminsession, row.artist, row.artist, row.song, row.firstname, row.lastname)

select itemInSession, artist, song, firstName, lastName from song_user_data_by_session where userid = 10 and sessionid = 182
0 Down To The Bone Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3 
#### SQL Requirement:
<p>Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'</p>

#### Select Query: 
<p>select firstName, lastName from user_data_by_song where song = 'All Hands Against His Own'</p>

- Table Name: user_data_by_song
- Primary Key: song, userid
- Partition Key: song
- Clustering Columns: userid
- Outout Cols: firstName, lastName

In [74]:
create_table_query3 = "CREATE TABLE IF NOT EXISTS user_data_by_song "
create_table_query3 = create_table_query3 + "(song text, userid int, firstName text, lastName text,PRIMARY KEY (song, userid))"
try:
    session.execute(create_table_query3)
except Exception as e:
    print(e)

    
insert_query3 = "INSERT INTO user_data_by_song (song, userid, firstName, lastName)"
insert_query3 = insert_query3 + " VALUES (%s, %s, %s, %s)"

query3 = "select firstName, lastName from user_data_by_song where song = 'All Hands Against His Own'"                   

In [75]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(insert_query3, (line[9], int(line[10]), line[1], line[4]))

- Composite Primary Key: song, userid
- Partition Key: song
- Clustering Key: userid
<br/> Partitioning is done by song and within that partition, rows ordered by the userid

In [76]:
try:
    print(query3)
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

select firstName, lastName from user_data_by_song where song = 'All Hands Against His Own'
Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [77]:
try:
    session.execute("drop table song_data_by_session")
    session.execute("drop table song_user_data_by_session")
    session.execute("drop table user_data_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [78]:
session.shutdown()
cluster.shutdown()
print("All steps are done...")

All steps are done...
